In [1]:
#IMPORTING scikit packages
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats


In [2]:
#IMPORTING SETUP PACKAGES
import pickle
import os.path
import os
import pandas as pd
import numpy as np

import psycopg2
import timeit
from datetime import datetime, timedelta, date
import datetime as dt
from calendar import monthrange
import random
import io
import boto3
import pyarrow as pa
import pyarrow.parquet as pq

from s3fs.core import S3FileSystem

import s3fs


%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 5000)

In [6]:
df = pd.read_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/data_model_cat_merged.pickle')

df

,item_code,update_date,delivery_weeks,item_price,sales,item_main_category,item_sub_category_1,item_sub_category_2,item_parent_item_code
0,000000001000000013,2020-01-30,1,39.99,0,GARTEN,GARTENZUBEHÖR,DEKORATION & ACCESSOIRES,000000001000000013-P
1,000000001000000013,2020-10-26,0,31.99,0,GARTEN,GARTENZUBEHÖR,DEKORATION & ACCESSOIRES,000000001000000013-P
2,000000001000000013,2020-10-27,0,31.99,0,GARTEN,GARTENZUBEHÖR,DEKORATION & ACCESSOIRES,000000001000000013-P
3,000000001000000014,2020-01-14,0,74.99,0,GARTEN,GARTENZUBEHÖR,DEKORATION & ACCESSOIRES,000000001000000014-P
4,000000001000000020,2020-03-30,1,74.99,0,GARTEN,GARTENZUBEHÖR,DEKORATION & ACCESSOIRES,000000001000000020-P
...,...,...,...,...,...,...,...,...,...
30447077,000000001000227044,2020-11-18,1,44.99,0,HEIMTEXTILIEN,KISSEN,DEKOKISSEN,000000008000031005
30447078,000000001000227056,2020-11-04,1,29.99,0,HEIMTEXTILIEN,KISSEN,KISSENBEZÜGE,000000001000227056-P
30447079,000000001000227092,2020-11-02,1,114.99,0,HEIMTEXTILIEN,TAGESDECKEN & PLAIDS,PLAIDS,000000001000227092-P
30447080,000000001000227855,2020-11-02,0,63.99,0,WOHNEN,TISCHE,COUCHTISCHE,000000008000013701


In [7]:
df.describe()

,delivery_weeks,item_price,sales
count,3.044708e+07,3.044708e+07,3.044708e+07
mean,3.326022e+00,3.711034e+02,7.267915e-02
std,4.625920e+00,4.814437e+02,5.059961e-01
min,-1.900000e+01,0.000000e+00,0.000000e+00
25%,0.000000e+00,5.999000e+01,0.000000e+00
50%,1.000000e+00,1.651300e+02,0.000000e+00
75%,5.000000e+00,4.999900e+02,0.000000e+00
max,2.010000e+02,7.599990e+03,1.210000e+02


In [8]:
print(len(df.columns))

9


In [10]:
df['log_price'] = np.log(df['item_price'])


In [ ]:
df

,item_code,update_date,delivery_weeks,item_price,sales,item_main_category,item_sub_category_1,item_sub_category_2,item_parent_item_code,log_price
0,000000001000000013,2020-01-30,1,39.99,0,GARTEN,GARTENZUBEHÖR,DEKORATION & ACCESSOIRES,000000001000000013-P,3.688629
1,000000001000000013,2020-10-26,0,31.99,0,GARTEN,GARTENZUBEHÖR,DEKORATION & ACCESSOIRES,000000001000000013-P,3.465423
2,000000001000000013,2020-10-27,0,31.99,0,GARTEN,GARTENZUBEHÖR,DEKORATION & ACCESSOIRES,000000001000000013-P,3.465423
3,000000001000000014,2020-01-14,0,74.99,0,GARTEN,GARTENZUBEHÖR,DEKORATION & ACCESSOIRES,000000001000000014-P,4.317355
4,000000001000000020,2020-03-30,1,74.99,0,GARTEN,GARTENZUBEHÖR,DEKORATION & ACCESSOIRES,000000001000000020-P,4.317355
...,...,...,...,...,...,...,...,...,...,...
30447077,000000001000227044,2020-11-18,1,44.99,0,HEIMTEXTILIEN,KISSEN,DEKOKISSEN,000000008000031005,3.806440
30447078,000000001000227056,2020-11-04,1,29.99,0,HEIMTEXTILIEN,KISSEN,KISSENBEZÜGE,000000001000227056-P,3.400864
30447079,000000001000227092,2020-11-02,1,114.99,0,HEIMTEXTILIEN,TAGESDECKEN & PLAIDS,PLAIDS,000000001000227092-P,4.744845
30447080,000000001000227855,2020-11-02,0,63.99,0,WOHNEN,TISCHE,COUCHTISCHE,000000008000013701,4.158727
